In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
bookings = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
comments = pd.read_csv("artifacts/data_ingestion/generated_data/comments.csv")
likes = pd.read_csv("artifacts/data_ingestion/generated_data/likes.csv")
bookings.head()
comments.head()
likes.head()

,user_id,liked_event_id,liked_event_organizer_id,like_id
0,1e896ff4-117b-4484-ac1d-fe3e753a4914,23f7e309-a99d-41a6-afa3-a05e33ba5432,b7a02932-0ce0-4769-9a3b-e4a398838ddf,b033d198-4984-4370-aa0c-c3d08399bde9
1,6d6c7236-649e-4b05-99e9-ddb085891243,03690412-d735-442b-bd09-8e55333c4ff1,853b4ab4-d45d-4fec-93cd-b74b044bb43a,6fa85722-896b-4bda-a8d3-dc70e82c7b90
2,b69ecd8b-a0cf-45fe-9916-3fe400c81cc9,74afaf8f-0e27-4e69-8a07-7795b4d2dca5,0a83494b-7db2-4f9d-9d77-58ec9cdb8c5f,ce741b3d-d137-4744-8ec4-4f00f9b954f7
3,e6cd18fa-dd5f-4cf5-b45c-d30d9c957085,95a4ee4c-f65d-422b-be57-ee954a9e19f3,918faa93-f824-4c15-8106-0a75a421f132,ff6fa94b-9d51-4b3f-b252-f1d7b0018c84
4,ad574736-b69c-44d9-9848-7902615012f5,20487871-ae5b-42f4-a1ed-ddd467748d6d,24b778a8-b330-4434-b505-3d9946db125c,079ebc49-6005-459b-9446-187d53443263


In [7]:
bookings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    150000 non-null  object
 1   booked_event_id            150000 non-null  object
 2   booked_event_organizer_id  150000 non-null  object
 3   booking_id                 150000 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [8]:
comments.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245397 entries, 0 to 245396
Data columns (total 4 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   user_id                       245397 non-null  object
 1   commented_event_id            245397 non-null  object
 2   commented_event_organizer_id  245397 non-null  object
 3   comment_id                    245397 non-null  object
dtypes: object(4)
memory usage: 7.5+ MB


In [9]:
likes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145397 entries, 0 to 145396
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   user_id                   145397 non-null  object
 1   liked_event_id            145397 non-null  object
 2   liked_event_organizer_id  145397 non-null  object
 3   like_id                   145397 non-null  object
dtypes: object(4)
memory usage: 4.4+ MB


In [10]:
# Standardize column names across all datasets
bookings = bookings.rename(columns={
    'booked_event_id': 'event_id',
    'booked_event_organizer_id': 'organizer_id',
    'booking_id': 'id'
})

comments = comments.rename(columns={
    'commented_event_id': 'event_id',
    'commented_event_organizer_id': 'organizer_id',
    'comment_id': 'id'
})

likes = likes.rename(columns={
    'liked_event_id': 'event_id',
    'liked_event_organizer_id': 'organizer_id',
    'like_id': 'id'
})

# Create source-specific DataFrames with indicator columns
bookings_df = bookings[['user_id', 'event_id', 'organizer_id']].copy()
bookings_df['is_booked'] = 1
bookings_df['num_bookings'] = 1

comments_df = comments[['user_id', 'event_id', 'organizer_id']].copy()
comments_df['is_commented'] = 1
comments_df['num_comments'] = 1

likes_df = likes[['user_id', 'event_id', 'organizer_id']].copy()
likes_df['is_liked'] = 1
likes_df['num_likes'] = 1

# Get all unique user-event pairs
unique_pairs = pd.concat([
    bookings[['user_id', 'event_id']],
    comments[['user_id', 'event_id']],
    likes[['user_id', 'event_id']]
]).drop_duplicates()

# Randomly select 100,000 unique pairs if we have more than that
if len(unique_pairs) > 200000:
    unique_pairs = unique_pairs.sample(n=200000, random_state=42)

# Get consistent organizer_id mapping (most frequent for each event)
organizer_mapping = pd.concat([
    bookings[['event_id', 'organizer_id']],
    comments[['event_id', 'organizer_id']],
    likes[['event_id', 'organizer_id']]
]).groupby('event_id')['organizer_id'].agg(lambda x: x.mode()[0]).to_dict()

# Merge all interaction types
result = unique_pairs.copy()

# Merge booking data
result = result.merge(
    bookings_df.groupby(['user_id', 'event_id']).agg({
        'is_booked': 'max',
        'num_bookings': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Merge comment data
result = result.merge(
    comments_df.groupby(['user_id', 'event_id']).agg({
        'is_commented': 'max',
        'num_comments': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Merge like data
result = result.merge(
    likes_df.groupby(['user_id', 'event_id']).agg({
        'is_liked': 'max',
        'num_likes': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Fill NA values with 0 for indicator and count columns
for col in ['is_booked', 'num_bookings', 'is_commented', 'num_comments', 'is_liked', 'num_likes']:
    result[col] = result[col].fillna(0).astype(int)

# Consolidate organizer_id (take first non-NA value)
result['organizer_id'] = (result['organizer_id_x']
                         .fillna(result['organizer_id_y'])
                         .fillna(result['organizer_id']))

# If still missing, use our mapping
result['organizer_id'] = result['organizer_id'].fillna(result['event_id'].map(organizer_mapping))

# Select final columns
final_data = result[[
    'user_id', 
    'event_id', 
    'is_booked',
    'num_bookings',
    'is_liked',
    'num_likes',
    'is_commented', 
    'num_comments', 
    'organizer_id'
]]

# Ensure we have exactly 100,000 records (in case some merges added duplicates)
final_data = final_data.drop_duplicates().sample(n=min(200000, len(final_data)), random_state=42)

# Save the merged data
final_data.to_csv("artifacts/data_ingestion/generated_data/merged_interactions.csv", index=False)

print("Merged data saved successfully!")
print(f"Final dataset shape: {final_data.shape}")
print(final_data.head())

Merged data saved successfully!
Final dataset shape: (200000, 9)
                                     user_id  \
119737  852d7b2a-de11-46a9-ae0b-d626440afe3d   
72272   f4b93c37-21b6-4772-a73d-180a30988e51   
158154  3187edcc-71d0-4e50-8e25-0915ef32ac8f   
65426   93a77014-edb4-4019-9f4b-bb6b249fa4c6   
30074   6d909574-9714-4e3f-939f-9d05a1d1337d   

                                    event_id  is_booked  num_bookings  \
119737  ebf7d8c2-2af0-432d-8b76-b82e9565cea0          0             0   
72272   6aa420d9-0d70-4f35-a3c3-3d6547e0a5de          0             0   
158154  b1e91aca-c168-4770-a5a8-e05f93b2a2c6          1             4   
65426   13c0e345-5370-4751-87e1-4a1d58dc99df          0             0   
30074   d1622d96-daba-4c09-a5b2-4d96647652c7          0             0   

        is_liked  num_likes  is_commented  num_comments  \
119737         0          0             1             1   
72272          0          0             1             3   
158154         0          0   